In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,324 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,075 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 http://secu

In [2]:
# Download a Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.17.jar

--2023-04-12 20:23:04--  https://jdbc.postgresql.org/download/postgresql-42.2.17.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1004734 (981K) [application/java-archive]
Saving to: ‘postgresql-42.2.17.jar’

postgresql-42.2.17. 100%[===================>] 981.19K  --.-KB/s    in 0.09s   

2023-04-12 20:23:04 (10.7 MB/s) - ‘postgresql-42.2.17.jar’ saved [1004734/1004734]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Module_Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.17.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [5]:
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [6]:
# Drop null values
dropna_df = df.dropna()


In [8]:
from pyspark.sql.functions import col
# Create the vine_table. DataFrame
vine_df = dropna_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df = vine_df.withColumn("star_rating",col("star_rating").cast("integer"))
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [9]:
# Load in a SQL function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
vine_df_filtered = vine_df.filter(col("total_votes") >= 20)
vine_df_filtered.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [10]:
clean_vine_df = vine_df_filtered.filter((vine_df_filtered.helpful_votes / vine_df_filtered.total_votes) >= 0.5)
clean_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [11]:
vine_yes = clean_vine_df.filter(col("vine") == 'Y')
vine_yes.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R32OMS1LSDRCOB|          4|           27|         31|   Y|                N|
|R2IPKK4I0XRCFR|          4|           16|         21|   Y|                N|
| RXYYJ8SGI9IX6|          5|           36|         41|   Y|                N|
|R1SAUBFC7T6JHC|          5|           73|         75|   Y|                N|
|R329HVJKKHWSG3|          5|           59|         59|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [12]:
vine_no = clean_vine_df.filter(col("vine") == 'N')
vine_no.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [27]:
# count of paid reviews (vine_yes)
paid_count = vine_yes.count()
print('Number of review for paying members:',paid_count)

Number of review for paying members: 334


In [26]:
# count of unpaid reviews (vine_no)
unpaid_count = vine_no.count()
print('Number of reviews for non-paying members:',unpaid_count)

Number of reviews for non-paying members: 61589


In [24]:
# count pf paid 5-star reviews
count_paid_fivestar = vine_yes.filter(col("star_rating") == 5).count()
print('Number of 5-star reviews for paying members:',count_paid_fivestar)

Number of 5-star reviews for paying members: 139


In [25]:
# count pf unpaid 5-star reviews
count_unpaid_fivestar = vine_no.filter(col("star_rating") == 5).count()
print('Number of 5-star reviews for non-paying members:',count_unpaid_fivestar)

Number of 5-star reviews for non-paying members: 32660


In [29]:
# paid 5_star review %
paid_fivestar_percent = round((count_paid_fivestar / paid_count)*100, 2)
print('5 star percentage for paying members:',paid_fivestar_percent)

5 star percentage for paying members: 41.62


In [28]:
# unpaid 5_star review %
unpaid_fivestar_percent = round((count_unpaid_fivestar / unpaid_count)*100, 2)
print('5 star percentage for non-paying memebers:',unpaid_fivestar_percent)

5 star percentage for non-paying memebers: 53.03
